<a href="https://colab.research.google.com/github/joker2017/Calculator/blob/master/Convolution_blocks_for_mobilenet.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:

"""Convolution blocks for mobilenet."""
import contextlib
import functools

import tensorflow as tf

slim = tf.contrib.slim


def _fixed_padding(inputs, kernel_size, rate=1):
  """Дополняет ввод по пространственным измерениям независимо от размера ввода.

  Дополняет ввод так, что если бы он использовался в свертке с заполнением 'VALID',
  выход имел бы те же размеры, как если бы незаполненный ввод использовался в свертке
  с заполнением 'SAME'.

  Args:
    inputs: A tensor of size [batch, height_in, width_in, channels].
    kernel_size: The kernel to be used in the conv2d or max_pool2d operation.
    rate: An integer, rate for atrous convolution.

  Returns:
    output: A tensor of size [batch, height_out, width_out, channels] with the
      input, either intact (if kernel_size == 1) or padded (if kernel_size > 1).
  """
  kernel_size_effective = [kernel_size[0] + (kernel_size[0] - 1) * (rate - 1),
                           kernel_size[0] + (kernel_size[0] - 1) * (rate - 1)]
  pad_total = [kernel_size_effective[0] - 1, kernel_size_effective[1] - 1]
  pad_beg = [pad_total[0] // 2, pad_total[1] // 2]
  pad_end = [pad_total[0] - pad_beg[0], pad_total[1] - pad_beg[1]]
  padded_inputs = tf.pad(inputs, [[0, 0], [pad_beg[0], pad_end[0]],
                                  [pad_beg[1], pad_end[1]], [0, 0]])
  return padded_inputs


def _make_divisible(v, divisor, min_value=None):
  if min_value is None:
    min_value = divisor
  new_v = max(min_value, int(v + divisor / 2) // divisor * divisor)
  # Make sure that round down does not go down by more than 10%.
  if new_v < 0.9 * v:
    new_v += divisor
  return new_v


def _split_divisible(num, num_ways, divisible_by=8):
  """Равномерно разбивает num, num_ways, поэтому каждый фрагмент кратен divisible_by."""
  assert num % divisible_by == 0
  assert num / num_ways >= divisible_by
  # Примечание: хотите округлить, мы корректируем каждый сплит в соответствии с итогом.
  base = num // num_ways // divisible_by * divisible_by
  result = []
  accumulated = 0
  for i in range(num_ways):
    r = base
    while accumulated + r < num * (i + 1) / num_ways:
      r += divisible_by
    result.append(r)
    accumulated += r
  assert accumulated == num
  return result


@contextlib.contextmanager
def _v1_compatible_scope_naming(scope):
  if scope is None:  # Create uniqified separable blocks.
    with tf.variable_scope(None, default_name='separable') as s, \
         tf.name_scope(s.original_name_scope):
      yield ''
  else:
    # Мы используем scope_depthwise, scope_pointwise для совместимости с v1 ckpts, которые предоставляют пронумерованные области.
    
    scope += '_'
    yield scope


@slim.add_arg_scope
def split_separable_conv2d(input_tensor,
                           num_outputs,
                           scope=None,
                           normalizer_fn=None,
                           stride=1,
                           rate=1,
                           endpoints=None,
                           use_explicit_padding=False):
  """Separable mobilenet V1 style convolution.

  Глубинная свертка, с нелинейностью по умолчанию,
  с последующей глубокой сверткой 1x1. Это похоже на
  slim.separable_conv2d, но отличается тем, что применяет пакет
  нормализация и нелинейность по глубине. Это соответствует
  основное здание Mobilenet Paper
  (https://arxiv.org/abs/1704.04861)

  Args:
    input_tensor: input
    num_outputs: количество выходов
    scope: необязательное имя области. Обратите внимание, что при наличии он будет использовать 
    scope_depthwise для deptwhise и scope_pointwise для pointwise. 
    normalizer_fn: какую функцию нормализатора использовать для depthwise/pointwise
    stride: stride
    rate: output rate (также известный как скорость расширения)
    endpoints: необязательно, если предоставлено, экспортирует дополнительные тензоры к нему.
    use_explicit_padding: Используйте заполнение 'VALID' для сверток, но вводите входные 
    данные таким образом, чтобы выходные размеры были такими же, как если бы использовалось 
    заполнение 'SAME'.

  Returns:
    output tesnor
  """

  with _v1_compatible_scope_naming(scope) as scope:
    dw_scope = scope + 'depthwise'
    endpoints = endpoints if endpoints is not None else {}
    kernel_size = [3, 3]
    padding = 'SAME'
    if use_explicit_padding:
      padding = 'VALID'
      input_tensor = _fixed_padding(input_tensor, kernel_size, rate)
    net = slim.separable_conv2d(
        input_tensor,
        None,
        kernel_size,
        depth_multiplier=1,
        stride=stride,
        rate=rate,
        normalizer_fn=normalizer_fn,
        padding=padding,
        scope=dw_scope)

    endpoints[dw_scope] = net

    pw_scope = scope + 'pointwise'
    net = slim.conv2d(
        net,
        num_outputs, [1, 1],
        stride=1,
        normalizer_fn=normalizer_fn,
        scope=pw_scope)
    endpoints[pw_scope] = net
  return net


def expand_input_by_factor(n, divisible_by=8):
  return lambda num_inputs, **_: _make_divisible(num_inputs * n, divisible_by)


@slim.add_arg_scope
def expanded_conv(input_tensor,
                  num_outputs,
                  expansion_size=expand_input_by_factor(6),
                  stride=1,
                  rate=1,
                  kernel_size=(3, 3),
                  residual=True,
                  normalizer_fn=None,
                  project_activation_fn=tf.identity,
                  split_projection=1,
                  split_expansion=1,
                  expansion_transform=None,
                  depthwise_location='expansion',
                  depthwise_channel_multiplier=1,
                  endpoints=None,
                  use_explicit_padding=False,
                  padding='SAME',
                  scope=None):
  """Depthwise Convolution Block with expansion.
  Блок глубокой свертки с расширением.


  Создает сложную свертку, которая имеет следующую структуру
  расширение (1x1) -> depthwise (kernel_size) -> projection (1x1)

  Args:
    input_tensor: input
    num_outputs: количество выходов в последнем слое.
    expansion_size: Размер расширения может быть постоянным или вызываемым.
    В последнем случае будет предоставлено num_inputs в качестве входных данных. Для прямой 
    совместимости он должен принимать произвольные аргументы ключевых слов. По умолчанию 
    будет расширять ввод в 6 раз.
    stride: depthwise stride
    rate: depthwise rate
    kernel_size: depthwise kernel
    residual: включить ли  residual connection(остаточную связь) между input и output.
    normalizer_fn: batchnorm или иное
    project_activation_fn: activation function for the project layer
    split_projection: how many ways to split projection operator
      (that is conv expansion->bottleneck)
    split_expansion: how many ways to split expansion op
      (that is conv bottleneck->expansion) ops will keep depth divisible
      by this value.
    expansion_transform: Необязательная функция, которая принимает расширение как один вход и возвращает вывод.
    depthwise_location: куда поместить глубинные значения covnvolutions, поддерживаемые None, 'input', 'output', 'extension'
    depthwise channel multiplier:
    each input will replicated (with different filters)
    that many times. So if input had c channels,
    output will have c x depthwise_channel_multpilier.
    endpoints: Необязательный словарь, в который помещаются промежуточные конечные точки. 
    Ключи "extension_output", "deepwise_output", "projection_output" и 
    "extension_transform" всегда заполняются, даже если соответствующие функции не вызываются.
    use_explicit_padding: Используйте заполнение 'VALID' для сверток, но вводите входные 
    данные таким образом, чтобы выходные размеры были такими же, как если бы использовалось заполнение 'SAME'.
    padding: Тип заполнения, если `use_explicit_padding` не установлен.
    scope: optional scope.

  Returns:
    Tensor of depth num_outputs
    Тензор глубины num_outputs

  Raises:
    TypeError: on inval
  """
  with tf.variable_scope(scope, default_name='expanded_conv') as s, \
       tf.name_scope(s.original_name_scope):
    prev_depth = input_tensor.get_shape().as_list()[3]
    if  depthwise_location not in [None, 'input', 'output', 'expansion']:
      raise TypeError('%r is unknown value for depthwise_location' %
                      depthwise_location)
    if use_explicit_padding:
      if padding != 'SAME':
        raise TypeError('`use_explicit_padding` should only be used with '
                        '"SAME" padding.')
      padding = 'VALID'
    depthwise_func = functools.partial(
        slim.separable_conv2d,
        num_outputs=None,
        kernel_size=kernel_size,
        depth_multiplier=depthwise_channel_multiplier,
        stride=stride,
        rate=rate,
        normalizer_fn=normalizer_fn,
        padding=padding,
        scope='depthwise')
    # b1 -> b2 * r -> b2
    #   i -> (o * r) (bottleneck) -> o
    input_tensor = tf.identity(input_tensor, 'input')
    net = input_tensor

    if depthwise_location == 'input':
      if use_explicit_padding:
        net = _fixed_padding(net, kernel_size, rate)
      net = depthwise_func(net, activation_fn=None)

    if callable(expansion_size):
      inner_size = expansion_size(num_inputs=prev_depth)
    else:
      inner_size = expansion_size

    if inner_size > net.shape[3]:
      net = split_conv(
          net,
          inner_size,
          num_ways=split_expansion,
          scope='expand',
          stride=1,
          normalizer_fn=normalizer_fn)
      net = tf.identity(net, 'expansion_output')
    if endpoints is not None:
      endpoints['expansion_output'] = net

    if depthwise_location == 'expansion':
      if use_explicit_padding:
        net = _fixed_padding(net, kernel_size, rate)
      net = depthwise_func(net)

    net = tf.identity(net, name='depthwise_output')
    if endpoints is not None:
      endpoints['depthwise_output'] = net
    if expansion_transform:
      net = expansion_transform(expansion_tensor=net, input_tensor=input_tensor)
    #Обратите внимание, что в отличие от расширения, у нас всегда есть проекция для получения желаемого выходного размера.
    net = split_conv(
        net,
        num_outputs,
        num_ways=split_projection,
        stride=1,
        scope='project',
        normalizer_fn=normalizer_fn,
        activation_fn=project_activation_fn)
    if endpoints is not None:
      endpoints['projection_output'] = net
    if depthwise_location == 'output':
      if use_explicit_padding:
        net = _fixed_padding(net, kernel_size, rate)
      net = depthwise_func(net, activation_fn=None)

    if callable(residual):  # custom residual/ таможенный остаток
      net = residual(input_tensor=input_tensor, output_tensor=net)
    elif (residual and
          # проверка шага гарантирует, что мы не добавляем остатки, когда пространственные измерения отсутствуют
          stride == 1 and
          # Depth matches/Глубина совпадений
          net.get_shape().as_list()[3] ==
          input_tensor.get_shape().as_list()[3]):
      net += input_tensor
    return tf.identity(net, name='output')


def split_conv(input_tensor,
               num_outputs,
               num_ways,
               scope,
               divisible_by=8,
               **kwargs):
  """Creates a split convolution.
  
  Разбиение свертки разделяет вход и выход на блоки 'num_blocks' примерно одинакового размера 
  каждый и соединяет только $ i $ -ый вход с выходом $ i $.

  Args:
    input_tensor: input tensor
    num_outputs: number of output filters
    num_ways: num blocks to split by./количество блоков для разделения
    scope: scope for all the operators.
    divisible_by: make sure that every part is divisiable by this./убедитесь, что каждая часть делится этим.
    **kwargs: will be passed directly into conv2d operator/будет передано непосредственно в conv2d оператор
  Returns:
    tensor
  """
  b = input_tensor.get_shape().as_list()[3]

  if num_ways == 1 or min(b // num_ways,
                          num_outputs // num_ways) < divisible_by:
    # Don't do any splitting if we end up with less than 8 filters
    # on either side.
    return slim.conv2d(input_tensor, num_outputs, [1, 1], scope=scope, **kwargs)

  outs = []
  input_splits = _split_divisible(b, num_ways, divisible_by=divisible_by)
  output_splits = _split_divisible(
      num_outputs, num_ways, divisible_by=divisible_by)
  inputs = tf.split(input_tensor, input_splits, axis=3, name='split_' + scope)
  base = scope
  for i, (input_tensor, out_size) in enumerate(zip(inputs, output_splits)):
    scope = base + '_part_%d' % (i,)
    n = slim.conv2d(input_tensor, out_size, [1, 1], scope=scope, **kwargs)
    n = tf.identity(n, scope + '_output')
    outs.append(n)
  return tf.concat(outs, 3, name=scope + '_concat')